In [2]:
# imports
import os, time, json
import tmdbsimple as tmdb 
import pandas as pd
from tqdm.notebook import tqdm_notebook 

In [16]:
def write_json(new_data, filename):
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [17]:
def get_movie_with_rating(movie_id):
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)

    # save the .info. .releases dictionaries
    info = movie.info()
    releases = movie.releases()

    # Loop through countries in releases 
    for c in releases['countries']:
        # if the country abbreviation == US
        if c['iso_3166_1'] == 'US':
            ## save a 'certification' key in info with the certification 
            info['certification'] = c['certification']
    return info

In [18]:
import json 
with open('D:/My Documents/GitHub/Stack-3/.secret/tmdb_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

In [19]:
import tmdbsimple as tmdb
tmdb.API_KEY = login['api-key']

In [32]:
get_movie_with_rating('tt0332280')

{'adult': False,
 'backdrop_path': '/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg',
 'belongs_to_collection': None,
 'budget': 29000000,
 'genres': [{'id': 10749, 'name': 'Romance'}, {'id': 18, 'name': 'Drama'}],
 'homepage': 'http://www.newline.com/properties/notebookthe.html',
 'id': 11036,
 'imdb_id': 'tt0332280',
 'original_language': 'en',
 'original_title': 'The Notebook',
 'overview': "An epic love story centered around an older man who reads aloud to a woman with Alzheimer's. From a faded notebook, the old man's words bring to life the story about a couple who is separated by World War II, and is then passionately reunited, seven years later, after they have taken different paths.",
 'popularity': 71.028,
 'poster_path': '/rNzQyW4f8B8cQeg7Dgj3n6eT5k9.jpg',
 'production_companies': [{'id': 12,
   'logo_path': '/iaYpEp3LQmb8AfAtmTvpqd4149c.png',
   'name': 'New Line Cinema',
   'origin_country': 'US'},
  {'id': 1565, 'logo_path': None, 'name': 'Avery Pix', 'origin_country': 'US'},
  {'id': 26

In [33]:
basics = pd.read_csv('Data/basics.csv')
basics

,Unnamed: 0,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,34802,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,61114,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,67666,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,86793,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,93930,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama
...,...,...,...,...,...,...,...,...,...,...
88541,10016149,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019.0,NaN,74,Drama
88542,10016544,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019.0,NaN,97,"Comedy,Drama,Fantasy"
88543,10016684,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama
88544,10016693,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller"


In [35]:
FOLDER = 'TMDB_Data/'
sorted(os.listdir(FOLDER))

[]

In [36]:
# Set the year to filter for 
YEAR = 2000 

# Create an empty list for saving errors
errors = []

In [37]:
# Define the JSON file to store results for the year
JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'


# Check if the JSON file exists
file_exists = os.path.isfile(JSON_FILE)

# If it does not exist: create it
if file_exists == False:
    print(f"Creating {JSON_FILE} for API results for year={YEAR}.")
    
    # save an empty dict with just "imdb_id" to the new json file.
    with open(JSON_FILE,'w') as f:
        json.dump([{'imdb_id':0}],f)

# If it exists, print a message
else:
    print(f'The file {JSON_FILE} already exists.')

Creating TMDB_Data/tmdb_api_results_2000.json for API results for year=2000.


In [40]:
# Filtering for movies from selected startYear 
df = basics.loc[basics['startYear'] == YEAR].copy()

# saving movie ids to list
movie_ids = df['tconst']
movie_ids.head()

8     tt0113026
9     tt0113092
11    tt0115937
12    tt0116391
13    tt0116628
Name: tconst, dtype: object

In [41]:
# Load existing data from json into a dataframe called 'previous_df'
previous_df = pd.read_json(JSON_FILE)
previous_df

,imdb_id
0,0


In [43]:
# filter out any ids that are alrady in the JSON_FILE
movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

In [44]:
# Loop through movie_ids_to_get with a tqdm progress bar 
for movie_id in tqdm_notebook(movie_ids_to_get, f"Movies from {'imdb_id'}"):

    # Attempt to retrieve then data for the movie id 
    try:
        temp = get_movie_with_rating(movie_id)
        write_json(temp, JSON_FILE)
        time.sleep(0.02)

    except Exception as e: 
        errors.append([movie_id, e])

Movies from imdb_id:   0%|          | 0/1505 [00:00<?, ?it/s]

In [47]:
print(f"- Total errors: {len(errors)}")

- Total errors: 223


In [46]:
final_year_df

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0113026,0.0,/vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg,None,10000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",,62127.0,en,The Fantasticks,...,49666.0,86.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Try to remember the first time magic happened,The Fantasticks,0.0,5.500,24.0,PG
2,tt0113092,0.0,None,None,0.0,"[{'id': 878, 'name': 'Science Fiction'}]",,110977.0,en,For the Cause,...,0.0,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The ultimate showdown on a forbidden planet.,For the Cause,0.0,4.958,12.0,
3,tt0116391,0.0,/krEZg9tb6blhc7sV6Us2ZGQ0gA.jpg,None,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",,442869.0,hi,Gang,...,0.0,165.0,"[{'english_name': 'Hindi', 'iso_639_1': 'hi', ...",Released,,Gang,0.0,5.000,2.0,NaN
4,tt0118694,0.0,/n4GJFGzsc7NinI1VeGDXIcQjtU2.jpg,None,150000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",,843.0,cn,花樣年華,...,14204632.0,99.0,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,"Feel the heat, keep the feeling burning, let t...",In the Mood for Love,0.0,8.110,2476.0,PG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1278,tt7029820,0.0,None,None,7000.0,[],,604889.0,en,Scream For Christmas,...,0.0,80.0,[],Released,,Scream For Christmas,0.0,2.000,1.0,
1279,tt7197642,0.0,None,None,0.0,"[{'id': 35, 'name': 'Comedy'}]",,872676.0,en,"Goodbye, Merry-Go-Round",...,0.0,90.0,[],Released,,"Goodbye, Merry-Go-Round",0.0,0.000,0.0,
1280,tt7631344,0.0,None,None,0.0,"[{'id': 27, 'name': 'Horror'}]",,108864.0,en,Urban Evil: Trilogy of Fear,...,0.0,86.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,Urban Evil: Trilogy of Fear,0.0,4.000,1.0,
1281,tt7631368,0.0,/sF0gUHE0YzZNXYugTB2LFxJIppf.jpg,None,10000000.0,"[{'id': 27, 'name': 'Horror'}]",,97186.0,en,"I, Vampire",...,0.0,85.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,"I, Vampire",0.0,6.400,4.0,NR


In [45]:
final_year_df = pd.read_json(JSON_FILE)
final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)